In [3]:
import os
import pandas as pd
from tqdm import tqdm
import re

In [4]:
character = 'Fry' # 'Barney' | 'Sheldon' | 'Harry' | 'Fry' | 'Vader' | 'Joey' | 'Phoebe' | 'Default'

In [5]:
source_dict = {
    'HIMYM':{
        'dataset_folder': 'Episodes',
    },
    'Futurama':{
        'dataset_folder': 'Episodes',
    },
    'Friends':{
        'dataset_folder': None,
    },
    'HP':{
        'dataset_folder': None,
    },
    'SW':{
        'dataset_folder': 'Scripts',
    },
    'TBBT':{
        'dataset_folder': 'Episodes',
    },
}

character_dict = {
    'Barney': {
        'source':'HIMYM',
        'delete_names':["Barney's Secretary",
                        'Marshall to Barney',
                        "Barney's mom",
                        'Ted, from seeing Barney',
                        'Lily, holding Barney',
                        'Marshall, on the phone with Barney',
                        "At Casa a pezzi. Barney is playing the piano.Ted's father",
                        'Marshall, to the girl Barney is talking to']
    },
    'Sheldon': {
        'source':'TBBT',
        'delete_names':[]
    },
    'Harry': {
        'source':'HP',
        'delete_names':[]
    },
    'Fry': {
        'source':'Futurama',
        'delete_names':['Mrs fry',
                        'Mr fry',
                        'Luck of the fryrish']
    },
    'Vader': {
        'source':'SW',
        'delete_names':["INT. DARTH VADER'S WINGMAN - COCKPIT"]
    },
    'Joey': {
        'source':'Friends',
        'delete_names':["Joeys Sisters",
                        'Joey\'s Date', 
                        "Joey's Look-A-Like", 
                        'Joeys Sister', 
                        "Joey's Doctor", 
                        "Joey's Hand Twin", 
                        'Joeys Date', 
                        'Joeys Grandmother']
    },
    'Phoebe': {
        'source':'Friends',
        'delete_names':['Amy turns around to Phoebe',
                        'Phoebe Waitress']
    },
    'Default':None
}


In [6]:
if character != 'Default':
    source = character_dict[character]['source']

In [7]:
# Mount google drive (for Colab only)
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
else:
    base_folder = os.getcwd()
    
in_folder = os.path.join(base_folder, "Data", 'Characters', character)
if not os.path.exists(in_folder):
    os.makedirs(in_folder)

# Preprocessing

In [14]:
# Open the dataset documents and store their data into a DataFrame
def load_dataset():
    def _load_himym_friends_tbbt_dataset(sources_folder):
        dataframe_rows = []
        # Get number of documents and their names
        documents_n = len(os.listdir(sources_folder))
        documents_names = os.listdir(sources_folder)
        # Loop over documents
        for i in tqdm(range(documents_n)):
            filename = documents_names[i]
            sources_label = filename[:-4]
            # Open document
            with open(os.path.join(sources_folder, filename), encoding="utf8") as file:
                # Loop over lines (= words)
                for line in file.readlines():
                        dataframe_row = {
                            "source": sources_label,
                            "line": line,
                        }
                        dataframe_rows.append(dataframe_row)
        # Build the dataframe from the words
        df = pd.DataFrame(dataframe_rows)
        return df
    def _load_futurama_dataset(sources_folder):
        futurama_txt = ''
        # Loop over documents
        for filename in tqdm(os.listdir(sources_folder)):
            futurama_txt += open(os.path.join(sources_folder, filename)).read()
            print(futurama_txt[:1000])
        # Split lines
        start_idx = 0
        end_idx = 0
        lines = []
        while start_idx < len(futurama_txt):
            start_idx = futurama_txt.find('<b>', end_idx)
            if start_idx == -1: # if no '<b>' is found, just save the rest
                lines.append(futurama_txt[end_idx:].replace('</b>',''))
                break
            elif start_idx != end_idx: # '<b>' is found
                lines.append(futurama_txt[end_idx+4:start_idx])
            end_idx = futurama_txt.find('</b>', start_idx)
            if end_idx == -1: # if no '</b>' is found, just save the rest
                lines.append(futurama_txt[start_idx:].replace('<b>',''))
                break
        lines.append(futurama_txt[start_idx+3:end_idx])
        print(lines[:10])
        df = pd.DataFrame(lines, columns=['line'])
        return df
    def _load_hp_dataset(sources_folder):
        sep = ';'
        df = None
        df_files = []
        for filename in os.listdir(sources_folder):
            df_files.append(pd.read_csv(os.path.join(sources_folder, filename), sep=sep).rename(columns = lambda x: x.lower()))
        df = pd.concat(df_files)
        df = df.rename(columns = {'character':'character', 'sentence':'line'})
        return df
    def _load_sw_dataset(source_folder):
        dataframe_rows = []
        # Get number of documents and their names
        documents_n = len(os.listdir(source_folder))
        documents_names = os.listdir(source_folder)
        # Loop over documents
        for i in tqdm(range(documents_n)):
            filename = documents_names[i]
            film_name = filename[:-4]
            # Open document
            with open(os.path.join(source_folder, filename)) as file:
                film_rows = []
                sentence = ""
                empty_line_allow = False
                between_numbers = False
                found_character = False
                for line in file.readlines():
                    if re.search(r"^[0-9]+.", line) != None: # Line is number followed by dot (page number)
                        pass
                    elif re.search(r"^[A-Z]{2,}", line) != None: # Line begins with an-all caps (a character)
                        sentence += line
                        found_character = True
                        empty_line_allow = True
                    elif line.isspace():
                        if empty_line_allow:
                            pass
                        else:
                            if found_character:
                                film_row = {
                                    "film": film_name,
                                    "line": sentence,
                                }
                                film_rows.append(film_row)
                                sentence = ""
                                found_character = False
                    elif found_character:
                        sentence += line
                        empty_line_allow = False
                dataframe_rows.extend(film_rows)
        # Build the dataframe from the words
        df = pd.DataFrame(dataframe_rows)
        return df
    ### Function starts here
    if character == 'Default':
        return None
    sources_subfolder = source_dict[source]['dataset_folder']
    if sources_subfolder:
        sources_folder = os.path.join(base_folder, "Data", "Sources", source, sources_subfolder)
    else:
        sources_folder = os.path.join(base_folder, "Data", "Sources", source)
    if source == 'HIMYM' or source == 'Friends' or source == 'TBBT':
        df = _load_himym_friends_tbbt_dataset(sources_folder)
    elif source == 'Futurama':
        df = _load_futurama_dataset(sources_folder)
    elif source == 'HP':
        df = _load_hp_dataset(sources_folder)
    elif source == 'SW':
        df = _load_sw_dataset(sources_folder)
    return df

In [15]:
# Execute creation of dataset
df = load_dataset()
if not isinstance(df, type(None)):
    print("Loaded Dataset!")
    print()
    print(df.head())
    print(df.count())

 62%|██████████████████████████████████████████████████▋                              | 45/72 [00:00<00:00, 436.91it/s]

<pre>

<b>                                        FUTURAMA
</b>
                                       Episode 101 

<b>                                   "SPACE PILOT 3000"
</b>
                                           By

                             David X. Cohen &amp; Matt Groening

                         Transcribed by Dave, The Neutral Planet

<b>                
</b>
               [Over the caption December 31st 1999 a crude spaceship flies 
               through space, cruising over and under planets and a man speaks.]
<b> 
</b><b>               
</b><b>               
</b><b>                                     MAN
</b>                         (voice-over) Space. It seems to go on 
                         and on forever. But then you get to 
                         the end and the gorilla starts throwing 
                         barrels at you.
<b> 
</b><b>               
</b>               [A planet opens up and a huge gorilla starts throwing barrels 
              

100%|█████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 356.44it/s]

<pre>

<b>                                        FUTURAMA
</b>
                                       Episode 101 

<b>                                   "SPACE PILOT 3000"
</b>
                                           By

                             David X. Cohen &amp; Matt Groening

                         Transcribed by Dave, The Neutral Planet

<b>                
</b>
               [Over the caption December 31st 1999 a crude spaceship flies 
               through space, cruising over and under planets and a man speaks.]
<b> 
</b><b>               
</b><b>               
</b><b>                                     MAN
</b>                         (voice-over) Space. It seems to go on 
                         and on forever. But then you get to 
                         the end and the gorilla starts throwing 
                         barrels at you.
<b> 
</b><b>               
</b>               [A planet opens up and a huge gorilla starts throwing barrels 
              

In [191]:
from operator import itemgetter

def process_dataset(df):
    def _process_himym_dataset(df):
        df = df[~df['line'].str.startswith("[")]
        df = df[~df['line'].str.startswith("(")]
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df = df[~df['line'].isnull()]
        df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
        df = df.dropna()
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'][df['line'].str.len() >= 2]
        df = df[~df['line'].isnull()]
        df = df.dropna()
        df = df.reset_index(drop=True)
        return df
    def _process_tbbt_dataset(df):
        df = df[~df['line'].str.startswith("[")]
        df = df[~df['line'].str.startswith("(")]
        df = df[~df['line'].str.startswith("Scene: ")]
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df = df[~df['line'].isnull()]
        df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
        df = df.dropna()
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'][df['line'].str.len() >= 2]
        df = df[~df['line'].isnull()]
        df = df.dropna()
        df = df.reset_index(drop=True)
        return df
    def _process_futurama_dataset(df):
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\[.*\]","")
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df['line'] = df['line'].str.replace(r"\<.*\>","")
        df['line'] = df['line'].str.replace("\n","")
        df = df[~df['line'].str.startswith("(")]
        df = df[~df['line'].str.startswith("[")]
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df['line'] = df['line'][df['line'].str.len() >= 2]
        df = df.dropna()
        df = df.reset_index(drop=True)
        df_rows = []
        for row in tqdm(range(len(df)-1)):
            if df['line'][row].isupper():
                print("H")
                df_row = {
                    'line': df['line'][row+1].strip()[:512],
                    'character': df['line'][row].strip().capitalize()
                }
                df_rows.append(df_row)
        df = pd.DataFrame(df_rows)
        df = df[df['character'].str.contains('Futurama')==False]
        df = df.reset_index(drop=True)
        return df
    def _process_friends_dataset(df):
        df = df[~df['line'].str.startswith("[")]
        df = df[~df['line'].str.startswith("(")]
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df = df[~df['line'].isnull()]
        df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
        df = df.dropna()
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'][df['line'].str.len() >= 2]
        df = df[~df['line'].isnull()]
        df = df.dropna()
        df = df[~(df['character'] == 'Written by')]
        df = df.reset_index(drop=True)
        return df
    def _process_sw_dataset(df):
        df = df[~df['line'].str.startswith("[")]
        df = df[~df['line'].str.startswith("(")]
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df[['character', 'line']] = df['line'].str.split("\n", 1, expand=True)
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df = df[~df['line'].isnull()]
        df = df[df['character'].str.split().apply(lambda l: len(l)) <= 6]
        df = df.reset_index(drop=True)
        return df
    def _process_hp_dataset(df):
        df['line'] = df['line'].str.strip()
        df['line'] = df['line'].str.replace(r"\(.*\)","")
        df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n"," ")
        df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]","")
        df = df[~df['line'].isnull()]
        df = df.dropna()
        df['line'] = df['line'].str.strip()
        df['character'] = [line.lower() for line in df['character']]
        df = df[~df['line'].isnull()]
        df = df.dropna()
        df = df.reset_index(drop=True)
        return df
    # Function starts here
    if character == 'Default':
        return None
    if source == 'HIMYM':
        df = _process_himym_dataset(df)
    elif source == 'Friends':
        df = _process_friends_dataset(df)
    elif source == 'Futurama':
        df = _process_futurama_dataset(df)
    elif source == 'TBBT':
        df = _process_tbbt_dataset(df)
    elif source == 'HP':
        df = _process_hp_dataset(df)
    elif source == 'SW':
        df = _process_sw_dataset(df)
    return df

In [192]:
df = process_dataset(df)
if not isinstance(df, type(None)):
    print("Processed Dataset into line-character format!")
    print()
    print(df.head())
    print(len(df))

100%|████████████████████████████████████████████████████████████████████████| 15279/15279 [00:00<00:00, 184088.69it/s]


KeyError: 'character'

In [188]:
if not isinstance(df, type(None)):
    char_names = [c for c in df['character'] if character.lower() in c.lower()]
    print("Characters contanining", character, ":", set(char_names), len(char_names))

KeyError: 'character'

In [171]:
if not isinstance(df, type(None)):
    char_names = set(char_names) - set(character_dict[character]['delete_names'])

In [172]:
if not isinstance(df, type(None)):
    df['character'] = df['character'].apply(lambda x: character if x in char_names else x)

In [173]:
if not isinstance(df, type(None)):
    print("Unique character names in dataset after name processing:", df['character'].unique())

Unique character names in dataset after name processing: ['Monica' 'Joey' 'Chandler' 'Phoebe' 'All' 'Ross' 'Rachel' 'Waitress'
 'Paul' 'Priest on TV' 'Frannie' 'Ross and Rachel' 'Customer' 'Marsha'
 'Carol' 'Chandler and Joey' 'Mrs. Geller' 'Mr. Geller' 'Susan' 'Barry'
 'Robbie' 'Bernice' 'Dr. Oberman' 'Paula' 'Alan' 'The Guys' 'Lizzie'
 'Guys' 'Leslie' 'Kiki' 'Joanne' 'Receptionist' 'Pizza Guy' 'Both' 'Kid'
 'Angela' 'Janice' 'Woman' 'Bob' 'Aurora' 'Director' 'Jill' 'Mr. Heckles'
 'Paolo' 'Shelley' 'Aunt Lillian' 'Nurse' 'Andrea' 'Lowell' 'Terry' 'Guy'
 'Girl' 'Max' 'David' 'Sandy' 'Fun Bobby' 'Dick Clark' 'Jay Leno'
 'Mrs. Bing' 'Coma Guy' 'Joey, Chandler, and Ross' 'Carol and Susan'
 'Joey and Chandler' 'Girls' 'Roger' 'Mr. Tribbiani' 'Ronni'
 'Mrs. Tribbiani' 'Lorraine' 'Kristin' 'Fireman No. 1' 'Fireman No. 2'
 'Fireman No. 3' 'Mrs. Tedlock' 'Celia' 'Steve' 'Ursula' 'Teacher' 'Helen'
 'Nina' 'Ross and Susan' 'Mr. Douglas' 'Jamie' 'Fran'
 'Rachel and Chandler' 'Mr.Heckles' 'Soothin

In [174]:
if not isinstance(df, type(None)):
    print("Remaining", character, "sentences:", len(df[df['character'] == character]))

Remaining Phoebe sentences: 7460


In [175]:
# NOTE: May consider feeding one sentence and one Sheldon reply or multiple sentences encoded with one Sheldon reply
def get_character(df, level=2):
    if character == 'Default':
        return None
    dataframe_rows = []
    idxs_character = df[df['character'] == character].index
    dataframe_rows = []
    for i in idxs_character:
        l = []
        l.append(df['line'][i])
        for j in range(0,level):
            line = max(i-j-1,0)
            l.append(df['line'][line])
        dataframe_rows.append(l)
    df = pd.DataFrame(dataframe_rows, columns=['response', 'context', 'context/0'])
    return df

df = get_character(df)

In [176]:
if not isinstance(df, type(None)):
    print(df.head())

                                            response  \
0                           Wait, does he eat chalk?   
1  Just, 'cause, I don't want her to go through w...   
2                                                No.   
3                                           Ooh! Oh!   
4                                   Fine!  Be murky!   

                                             context  \
0  All right Joey, be nice.  So does he have a hu...   
1                           Wait, does he eat chalk?   
2                              Never had that dream.   
3                                            Thanks.   
4  No, no don't! Stop cleansing my aura! No, just...   

                                           context/0  
0  C'mon, you're going out with the guy! There's ...  
1  All right Joey, be nice.  So does he have a hu...  
2                                      That's right.  
3                        Let me get you some coffee.  
4                                           Ooh! Oh

In [177]:
if not isinstance(df, type(None)):
    print("Preprocessed dataset length:", len(df))

Preprocessed dataset length: 7460


In [178]:
if not isinstance(df, type(None)):
    char_path = os.path.join(base_folder, "Data", "Characters", character)
    if not os.path.exists(char_path):
        os.makedirs(char_path)
    df.to_csv(os.path.join(char_path, str(character)+".csv"), index=False)
    print("Saved dataset at", os.path.join(char_path, str(character)+".csv"))

Saved dataset at D:\University\Esami da Superare\Natural Language Processing\BarneyBot\BarneyBot\Data\Characters\Phoebe\Phoebe.csv
